In [1]:
import re
import os
import pandas as pd
import numpy as np
import sys
import jieba
from sklearn import metrics
from sklearn import preprocessing

In [2]:
print(sys.version)

3.8.2 (v3.8.2:7b3ab5921f, Feb 24 2020, 17:52:18) 
[Clang 6.0 (clang-600.0.57)]


In [3]:
os.chdir(r'/Replication_TrainingDatasetCensorship/') # set working directory

In [4]:
jieba.load_userdict("auxiliary/Dict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/8g/74cy_pd90b94x2b6lxyf92v40000gn/T/jieba.cache
Loading model cost 0.709 seconds.
Prefix dict has been built successfully.


In [5]:
words =[#start of freedom
    "自由", "言论自由", "集会自由", "新闻自由", "结社自由", "自由权", "民主自由", "自由言论", "创作自由", 
    "婚姻自主", "自由民主", "自由市场", "自决", "自决权", "生而自由", "自由自在", "自由选择", "自由思想", 
    "公民自由", "自由竞争", "宗教自由", "自由价格", 
    # democracy
    "民主", "自由民主", "民主自由", "民主制度", "民主化", "社会民主主义", "民主运动", "民主主义", "民主改革", 
    "民主制", "民主选举", "民主权力", "多党制", "民主法制", "民主权利" ,
    # election
    "选举", "直接选举", "议会选举", "间接选举", "直选", "换届选举", "民选", "投票选举", "全民公决", "总统大选", 
    "大选", "普选", "全民投票", "民主选举",
    # collective action
    "抗议", "示威", "示威游行", "示威抗议", "游行示威", "静坐示威", "绝食抗议", "请愿", "示威运动", "游行", 
    "罢教", "静坐", "集会游行", "罢课", "签名运动", 
    # negative figures
    "林彪", "王洪文", "张春桥", "江青", "姚文元", "刘晓波", "丹增嘉措", "李洪志", "陈水扁", "黎智英", "艾未未", 
    "李登辉", "李柱铭", "何俊仁", "陈方安生", "达赖", "魏京生",
    # social control
    "维稳", "处突", "社会治安", "反恐怖", "公安工作", "预防犯罪", "收容审查", "治安工作", "大排查", "扫黄打非", 
    "接访", "反邪教",
    # surveillance
    "监控", "监测", "监视", "管控", "监看", "监视系统", "截听", "监控中心", "情报服务", "排查", "监视器",
    "情报搜集", "间谍卫星", "管理网络", "监控器", "监控站", "监控室", "数据采集", 
    # CCP
    "党中央",  "中国共产党", "党支部", "中共中央", "共青团", "共青团中央", "党委", "中央党校",
    # historical events
    "抗日战争", "解放战争", "抗美援朝", "改革开放", "香港回归", "长征", "三大战役", "秋收起义", "南昌起义", "志愿军", 
    "土地改革", "六四", "遵义会议", "广州起义", "百团大战", "文革", "文化大革命", "大跃进", "四人帮",
    # positive figures
    "毛泽东", "江泽民", "胡锦涛", "习近平", "周恩来", "朱镕基", "温家宝", "李克强", "邓小平", "曾庆红", "华国锋", "李鹏", 
    "杨尚昆", "谷牧", "吴邦国", "李岚清", "纪登奎", "乔石","邹家华", "李瑞环", "俞正声", "张高丽", "田纪云", "回良玉", 
    "李源潮", "贾庆林", "姚依林", "张立昌", "尉健行", "姜春云", "李铁映", "王兆国", "罗干", "杨汝岱", "王光英", "彭佩云", 
    "刘云山", "丁关根", "彭真", "胡启立", "曾培炎", "何东昌"]

In [6]:
news = pd.read_csv('auxiliary/Political_news_titles.csv')
news = news[news['label'].isnull()==False]
punct = ['......', '，', '--', '：',  '丨', '「', '」', '、', '——',  '-',  '_', '...', '·', '。',
        '【', '】', '《', '》', '/', '(图)', '(组图)', '[图]', '‧', '.', ':', ',', '（下）', '（上）', '（图）', '（组图）', ' ',
        '丨', '|', '(', ')', '（', '）', '[', ']', "'", '’', '‘', '“', '”', '%', '?', '!']
for i in range(len(news)):
    text = news.iloc[i,0]
    for p in punct:
      text = text.replace(p, "")
    text = re.sub('[a-zA-Z]','',text)
    text = re.sub("\d+", "", text)
    news.iloc[i,0] = text
news = news[news['label']!=0]
news.drop_duplicates(subset=['sentence'], inplace=True)
news = news.dropna(how='any',axis=0) 
news = news.reset_index(drop=True)
news["label"] = news["label"].astype(int)
news.label.unique()
test_lab = news["label"]

In [18]:
# store accuracy
wk = []
rm = []
bd = []

In [8]:
for i in range(len(news)):
    news.iloc[i, 0] = ' ' + ' '.join(jieba.lcut(news.iloc[i, 0]))+' '
news.head()

,sentence,label
0,达里奥 资本 市场 已 不再 自由 贫富差距 达 年代 以来 最 严重,-1
1,新闻自由 日 联合国 秘书长 强调 记者 为 新冠大 流行 错误信息 提供,-1
2,一组 老照片 自由女神 像是 如何 诞生 的,1
3,在 互联网 自由 日 抗议 苹果公司 同 中共 合作 审查 网络,-1
4,香港 居民 权利 自由 的 坚实 保障,1


In [9]:
for i in range(1, 11):
    f = r'output/bd' + str(i) + '.txt'
    y_pred = np.loadtxt(f, comments="#", delimiter=",", unpack=False)
    bd.append(metrics.accuracy_score(test_lab, y_pred))
    res = pd.DataFrame(index=range(5000), columns=range(20))
    category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
                range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
    for n, c in enumerate(category):
        counter = 0
        for w in c:
            counter += 1
            word = ' ' + words[w] + ' '
            id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
            dif = (y_pred[id] - test_lab[id])
            start = 5000 - res.iloc[:,n].isnull().sum()
            end = start + len(dif.tolist())
            res.iloc[start:end, n] = dif.tolist()
            res.iloc[start:end, n+10] = counter
    outf = r'output/bd'+ str(i) + '.csv'
    res.to_csv(outf, header=False, index=False)

In [10]:
for i in range(1, 11):
    f = r'output/wk' + str(i) + '.txt'
    y_pred = np.loadtxt(f, comments="#", delimiter=",", unpack=False)
    wk.append(metrics.accuracy_score(test_lab, y_pred))
    res = pd.DataFrame(index=range(5000), columns=range(20))
    category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
                range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
    for n, c in enumerate(category):
        counter = 0
        for w in c:
            counter += 1
            word = ' ' + words[w] + ' '
            id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
            dif = (y_pred[id] - test_lab[id])
            start = 5000 - res.iloc[:,n].isnull().sum()
            end = start + len(dif.tolist())
            res.iloc[start:end, n] = dif.tolist()
            res.iloc[start:end, n+10] = counter
    outf = r'output/wk'+ str(i) + '.csv'
    res.to_csv(outf, header=False, index=False)

In [11]:
for i in range(1, 11):
    f = r'output/rm' + str(i) + '.txt'
    y_pred = np.loadtxt(f, comments="#", delimiter=",", unpack=False)
    rm.append(metrics.accuracy_score(test_lab, y_pred))
    res = pd.DataFrame(index=range(5000), columns=range(20))
    category = [range(0, 22), range(22, 37), range(37, 51), range(51, 66), range(66,83), 
                range(83, 95), range(95, 113), range(113, 121), range(121, 140), range(140, 182)]
    for n, c in enumerate(category):
        counter = 0
        for w in c:
            counter += 1
            word = ' ' + words[w] + ' '
            id = [i for i in range(len(news)) if word in news.iloc[i, 0]]
            dif = (y_pred[id] - test_lab[id])
            start = 5000 - res.iloc[:,n].isnull().sum()
            end = start + len(dif.tolist())
            res.iloc[start:end, n] = dif.tolist()
            res.iloc[start:end, n+10] = counter
    outf = r'output/rm'+ str(i) + '.csv'
    res.to_csv(outf, header=False, index=False)